<a href="https://colab.research.google.com/github/Kushal190903/unreliable-news-detection/blob/main/unreliable_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langchain_huggingface
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.2 MB/s eta 0:00:00


In [41]:
import pandas as pd
import re
import string
import numpy as np
# from time import perf_counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
# from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Dropout
from keras.regularizers import l2
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics import classification_report,confusion_matrix
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [4]:
dataframe1 = pd.read_csv("/content/drive/MyDrive/fake news detection files/train.csv")
dataframe1=dataframe1[['text','label']]
dataframe2_fake = pd.read_csv("/content/drive/MyDrive/fake news detection files/faken.csv")
dataframe2_true = pd.read_csv("/content/drive/MyDrive/fake news detection files/true.csv")
dataframe2_true["label"] = 0
dataframe2_fake["label"] = 1
# print(dataframe2_true.head(3))
# print(dataframe2_fake.head(3))
dataframe2_merge = pd.concat([dataframe2_fake, dataframe2_true], axis=0)

dataframe2 = dataframe2_merge.drop(["title", "subject","date"], axis=1)

# dataframe2.drop(["index"], axis=1, inplace=True)
dataframe=pd.concat([dataframe1,dataframe2],axis=0)
dataframe=dataframe.sample(frac=1)
dataframe.reset_index(inplace=True)
dataframe.drop(["index"], axis=1, inplace=True)
dataframe

,text,label
0,"After they were crushed in the 2012 election, ...",1
1,Fox News host Kimberly Guilfoyle has suggested...,0
2,BUENOS AIRES (Reuters) - Argentine President M...,0
3,Trump supporter gets revenge on lawn-sign vand...,1
4,AUDIO OF KARL ROVE ON HUGH HEWITT Picking ...,1
...,...,...
65714,.@SpeakerRyan the women of America are watchin...,1
65715,Liberal Tom “Rhymes With Liar” Steyer has be...,0
65716,"The driver pushed his way through the crowd, w...",1
65717,Students at a North Carolina high school have ...,1


In [5]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65719 entries, 0 to 65718
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    65680 non-null  object
 1   label   65719 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [64]:
print(dataframe.iloc[1]['text'])

Fox News host Kimberly Guilfoyle has suggested she would be interested in replacing Sean Spicer as White House press secretary, amid reports Spicer could soon be released from the role. [In an interview with Mercury News, Guilfoyle said: “I’m a patriot and it would be an honor to serve the country. I think it’d be a fascinating job. It’s a challenging job, and you need someone really determined and focused, a great communicator in there with deep knowledge to be able to handle that position. ”  Earlier this week, The New York Times reported that President Trump was considering Guilfoyle as a replacement for Spicer, whose time as press secretary has been marked by a number of gaffes and mistakes. An anonymously sourced report by ABC News also alleges that Trump is furious with his communications team over recent media narratives and could fire both Spicer and his communications director Mike Dubke. Guilfoyle, 48, is the former first lady of San Fransisco, having been married to former c

##wordopt can be used to filter the text currently not in use

In [77]:

def wordopt(t):
     t = t.lower()
     t = re.sub('\[.*?\]', 'bracket', t)
     t = re.sub("\\W"," ",t)
     t = re.sub('https?://\S+|www\.\S+', 'hyperlink', t)
     t = re.sub('<.*?>+', 'HTML', t)
     t = re.sub('[%s]' % re.escape(string.punctuation), '', t)
     t = re.sub('\n', '', t)
     #t = re.sub('\w*\d\w*', '', t)
     return t.strip()

In [78]:
dataframe=dataframe.dropna()

In [79]:
x=dataframe['text']
y=dataframe['label']
x=x.apply(wordopt)
# tokenizer=Tokenizer()
# tokenizer.fit_on_texts(x)
# x=tokenizer.texts_to_sequences(x)
# x=pad_sequences(x,maxlen=1000,padding='post',truncating='post')

embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
x=embedding.embed_documents(x)

In [80]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [81]:
len(x_train[0])

384

In [82]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=384))
model.add(Dense(32,activation='relu'))



model.add(Dense(32,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))

model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [83]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [84]:
model.fit(np.array(x_train),np.array(y_train),epochs=10,batch_size=32,validation_data=(np.array(x_val),np.array(y_val)))

Epoch 1/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7826 - loss: 0.5534 - val_accuracy: 0.8664 - val_loss: 0.3219
Epoch 2/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8700 - loss: 0.3213 - val_accuracy: 0.8806 - val_loss: 0.2913
Epoch 3/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8810 - loss: 0.2932 - val_accuracy: 0.8821 - val_loss: 0.2827
Epoch 4/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8949 - loss: 0.2657 - val_accuracy: 0.8905 - val_loss: 0.2684
Epoch 5/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9009 - loss: 0.2510 - val_accuracy: 0.8949 - val_loss: 0.2615
Epoch 6/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9073 - loss: 0.2397 - val_accuracy: 0.8940 - val_loss: 0.2603
Epoch 7/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9127 - loss: 0.2249 - val_accuracy: 0.8956 - val_loss: 0.2569
Epoch 8/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9174 - loss: 0.2151 - 

In [85]:
y_predict=model.predict(np.array(x_test))
y_pred=np.where(y_predict>0.5,1,0)

411/411 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [86]:
classificationreport=classification_report(y_test,y_pred)

In [87]:
print(classificationreport)

              precision    recall  f1-score   support

           0       0.91      0.88      0.89      6277
           1       0.89      0.92      0.91      6859

    accuracy                           0.90     13136
   macro avg       0.90      0.90      0.90     13136
weighted avg       0.90      0.90      0.90     13136



In [88]:
confusion_matrix(y_test,y_pred)

array([[5519,  758],
       [ 555, 6304]])

In [89]:
xgbclassifier=xgb.XGBClassifier(objective='binary:logistic',random_state=42)
xgbclassifier.fit(np.array(x_train),np.array(y_train))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [90]:
confusion_matrix(y_test,xgbclassifier.predict(x_test))

array([[5443,  834],
       [ 762, 6097]])

In [92]:
def classify(text):
    '''classify the text'''
    text = wordopt(text)
    embedded_text = np.array(embedding.embed_query(text)).reshape(1, -1)
    prediction = model.predict(embedded_text)
    return prediction[0][0]

In [94]:
text=input("enter the text")
classify(text)

enter the textGermany increases visa quota for skilled Indian professionals to 90,000 annually: PM Narendra Modi Prime minister Modi engaged in bilateral talks with German Chancellor Olaf Scholz today Prime Minister Narendra Modi on Friday announced that Germany has decided to increase the number of visas granted annually to skilled Indian workers to 90,000 from 20,000, reported news agency AFP. Modi is hosting German Chancellor Olaf Scholz for three days, during which the two engaged in bilateral talks concerning trade between the two countries, which constitute the world's fifth and third largest economies respectively. Also Read: Delhi-Berlin partnership has kept pace with India’s rise "Today Indians are the largest group of foreign students at our universities. Last year alone, the number of Indians working in Germany grew by 23,000. That talent is a welcome addition to our labour market," Scholz said while addressing the 18th Asia-Pacific Conference of German Business 2024. Also R

0.053063598

In [98]:
model.save('news_classifier_neural_network_model.keras')


In [99]:
from google.colab import files
files.download('news_classifier_neural_network_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [101]:
xgbclassifier.save_model('news_classifier_xgb.json')

In [102]:
files.download('news_classifier_xgb.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>